In [83]:
# Import necessary libraries
import os
import re
import pandas as pd
import numpy
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.models import CoherenceModel

# Preprocessing

In [84]:
# Function to load data from folder path
def load_data(folder_path):
    articles = []
    
	# Iterate through all files in the folder path
    for file in os.listdir(folder_path):
        with open(os.path.join(folder_path, file), "r", encoding="latin") as f:
            content = f.read()
            articles.append(content)
    return articles

In [85]:
# Function to preprocess the loaded data
def preprocess_data(article_files):
    cleaned_articles = []
     # Iterate through all articles in the input list   

    for article_file in article_files:
        # Split individual articles
        articles = article_file.split('')
        for article in articles:
	        # Remove new line characters
                text = re.sub(r"\s+", " ", article)
                # Remove numbers and punctuation
                text = re.sub(r"[^a-zA-Z\s]", "", text)
                # Make all characters lowercase
                text = text.lower()
                # Tokenize text into words
                tokens = word_tokenize(text)
	        # Remove stop words (common words with little meaning such as "the" or "a")
                stop_words = set(stopwords.words("english"))
                custom_stopwords = {"mr", "said", "u"}  # Add any additional stopwords here
                stop_words = stop_words.union(custom_stopwords)
	        # Remove any remaining short words and lemmatize 
                tokens = [token for token in tokens if token not in stop_words]
                tokens = [token for token in tokens if len(token) > 1 or token in {"i", "a"}]
                lemmatizer = WordNetLemmatizer()
                tokens = [lemmatizer.lemmatize(token) for token in tokens]
                tokens = [token for token in tokens if token != "u"]
                cleaned_articles.append(tokens)
    return cleaned_articles

In [86]:
# Function to create a corpus and dictionary from the preprocessed data
def create_corpus_dictionary(cleaned_articles):
    dictionary = corpora.Dictionary(cleaned_articles)
    corpus = [dictionary.doc2bow(article) for article in cleaned_articles]
    return corpus, dictionary

In [87]:
# Run preprocessing functions
folder_path = "Articles/"
article_files = load_data(folder_path)
cleaned_articles = preprocess_data(article_files)
corpus, dictionary = create_corpus_dictionary(cleaned_articles)

# Topic Models

In [88]:
# Function to train an LDA model
def train_lda(corpus, dictionary, num_topics, passes):
    lda_model = models.LdaModel(corpus=corpus,
                                num_topics=num_topics,
                                id2word=dictionary,
                                passes=passes,
                                random_state=13)
    return lda_model

In [89]:
# Function to print and plot the topics from an LDA model
def plot_topics(lda_model):
    num_topics = lda_model.num_topics
    topic_words = lda_model.print_topics(num_words=10)
    
    for topic in range(num_topics):
        print(f"Topic {topic}:")
        print(topic_words[topic])

In [90]:
# Function to run the LDA training and save the results to a file
def run_and_save_lda(corpus, dictionary, num_topics, passes, model_number, output_file):
    lda_model = train_lda(corpus, dictionary, num_topics, passes)

    # save the 4 models to Models/ folder
    temp_file = 'Models/lda_model_' + str(model_number+1)
    lda_model.save(temp_file)

    # Save the topic words to output files
    topic_words = lda_model.print_topics(num_words=10)
    with open(output_file, "w") as f:
        for topic in range(num_topics):
            f.write(f"Topic {topic}:\n")
            f.write(str(topic_words[topic]) + "\n\n")

In [91]:
# Define different parameters to build the LDA model
# use different topic and pass count and discuss differences in report
params = [{"num_topics": 5, "passes": 10},{"num_topics": 10, "passes": 10},{"num_topics": 5, "passes": 20},{"num_topics": 10, "passes": 20}]


# Iterating through the parameters and building different LDA models
for i, p in enumerate(params):
    output_file = f"output_{i + 1}.txt"
    run_and_save_lda(corpus, dictionary,  p["num_topics"], p["passes"], i, output_file)

# Evaluation

In [92]:
# Load models from Models folder
model_1 = models.ldamodel.LdaModel.load("Models/lda_model_1")
model_2 = models.ldamodel.LdaModel.load("Models/lda_model_2")
model_3 = models.ldamodel.LdaModel.load("Models/lda_model_3")
model_4 = models.ldamodel.LdaModel.load("Models/lda_model_4")
models_list = [model_1, model_2, model_3, model_4]

In [93]:
# Function to print Coherence Score of model
def compute_coherence(lda_model):
    coherence_lda_model = CoherenceModel(model=lda_model, texts=cleaned_articles, dictionary=dictionary, coherence='c_v')
    coherence_number = coherence_lda_model.get_coherence()
    print('Coherence:  ', coherence_number)

In [94]:
model_number = 0

# Compute Perplexity of all 4 models
for m in models_list:
    model_number+=1
    print('Model ', model_number)
    print('Perplexity: ', m.log_perplexity(corpus))
    compute_coherence(m)
    print('\n')

Model  1
Perplexity:  -8.359674162028881
Coherence:   0.39373667344733476


Model  2
Perplexity:  -8.357186043023537
Coherence:   0.3805207742184284


Model  3
Perplexity:  -8.344430869720606
Coherence:   0.40365820143500164


Model  4
Perplexity:  -8.332313718670523
Coherence:   0.40780882606523905




# Visualization

In [95]:
# Enable Python Notebook display of pyLDAvis
pyLDAvis.enable_notebook()


In [96]:

# Model 1 visualization
vis = pyLDAvis.gensim_models.prepare(model_1, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.098613  0.064994       1        1  28.589859
1      0.095514  0.043401       2        1  19.464349
4     -0.082408 -0.080818       3        1  18.918401
3      0.078534 -0.058744       4        1  17.300381
2      0.006972  0.031166       5        1  15.727010, topic_info=          Term         Freq        Total Category  logprob  loglift
481      trump  7372.000000  7372.000000  Default  30.0000  30.0000
231    islamic  6041.000000  6041.000000  Default  29.0000  29.0000
44      attack  3600.000000  3600.000000  Default  28.0000  28.0000
40324    gigot  1026.000000  1026.000000  Default  27.0000  27.0000
952      force  3479.000000  3479.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
256       like   452.747545  2506.603410   Topic5  -5.9095   0.1384
312        one   501.748310  4140.694251   Topic5  -5.8067  -0.2607
819       iraq   447.322468  3026.044383   Topic5  -5.9215  -0.0619
329     people   452.930702  3908.894819   Topic5  -5.9091  -0.3055
391      right   428.547089  3224.145960   Topic5  -5.9644  -0.1682

[473 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
897        1  0.943152       abadi
897        2  0.005483       abadi
897        3  0.005483       abadi
897        4  0.005483       abadi
897        5  0.049351       abadi
...      ...       ...         ...
44617      4  0.966604       zahia
33149      3  0.984946      zarrab
4617       1  0.007600  zuckerberg
4617       2  0.007600  zuckerberg
4617       4  0.980370  zuckerberg

[1302 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 5, 4, 3])

In [97]:
# Model 2 visualization
vis = pyLDAvis.gensim_models.prepare(model_2, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.088185 -0.095154       1        1  23.965113
8     -0.051998 -0.021895       2        1  14.260686
1     -0.126087 -0.065175       3        1  14.037098
2     -0.020405 -0.085882       4        1   8.956973
6      0.047580  0.041831       5        1   8.430442
9     -0.039946  0.072044       6        1   8.299533
5      0.079868 -0.023455       7        1   7.740426
4      0.120714  0.023469       8        1   6.676551
3     -0.108817  0.037349       9        1   6.160756
7      0.010906  0.116868      10        1   1.472423, topic_info=        Term          Freq         Total Category  logprob  loglift
481    trump   7473.000000   7473.000000  Default  30.0000  30.0000
436    state  10048.000000  10048.000000  Default  29.0000  29.0000
231  islamic   6088.000000   6088.000000  Default  28.0000  28.0000
44    attack   3554.000000   3554.000000  Default  27.0000  27.0000
295      new   5942.000000   5942.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
312      one     35.289754   4134.277987  Topic10  -6.0927  -0.5452
391    right     34.558147   3247.466554  Topic10  -6.1137  -0.3247
481    trump     35.517776   7473.671406  Topic10  -6.0863  -1.1308
820      isi     32.747349   2733.367767  Topic10  -6.1675  -0.2062
156  english     31.987924   1648.109386  Topic10  -6.1910   0.2762

[884 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
897        1  0.911839       abadi
897        4  0.037327       abadi
897        7  0.042659       abadi
25467      7  0.922428     abballa
898        1  0.114126       abbas
...      ...       ...         ...
36487      5  0.921320      ziadeh
32291      9  0.903635        zuck
4617       1  0.007964  zuckerberg
4617       2  0.031854  zuckerberg
4617       9  0.947663  zuckerberg

[3306 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 9, 2, 3, 7, 10, 6, 5, 4, 8])

In [98]:
# Model 3 visualization
vis = pyLDAvis.gensim_models.prepare(model_3, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.120383  0.060349       1        1  27.875761
1      0.099606  0.059534       2        1  19.254812
4     -0.078020 -0.106403       3        1  19.039933
3      0.098219 -0.071268       4        1  17.993203
2      0.000578  0.057787       5        1  15.836291, topic_info=         Term          Freq         Total Category  logprob  loglift
481     trump   7393.000000   7393.000000  Default  30.0000  30.0000
231   islamic   6054.000000   6054.000000  Default  29.0000  29.0000
952     force   3499.000000   3499.000000  Default  28.0000  28.0000
436     state  10017.000000  10017.000000  Default  27.0000  27.0000
973     iraqi   2177.000000   2177.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
277  military    575.873232   3232.856300   Topic5  -5.6758   0.1176
22       also    542.603579   3023.331786   Topic5  -5.7353   0.1251
819      iraq    454.685854   3046.710433   Topic5  -5.9121  -0.0593
312       one    451.449700   4131.566636   Topic5  -5.9193  -0.3711
391     right    444.221794   3224.587315   Topic5  -5.9354  -0.1394

[468 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
897        1  0.968224       abadi
897        5  0.027197       abadi
19681      1  0.005503       abedi
19681      3  0.990613       abedi
40293      1  0.972985    abihabib
...      ...       ...         ...
5010       1  0.991601         ypg
5010       5  0.006746         ypg
33149      3  0.977277      zarrab
4617       2  0.007687  zuckerberg
4617       4  0.983980  zuckerberg

[1206 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 5, 4, 3])

In [99]:
# Model 4 visualization
vis = pyLDAvis.gensim_models.prepare(model_4, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.118662  0.073018       1        1  23.567547
1      0.112371  0.104266       2        1  14.145337
8      0.048985  0.035070       3        1  13.421787
2     -0.006873  0.106371       4        1   9.696319
9      0.067574 -0.040026       5        1   9.101092
6     -0.051984 -0.045742       6        1   8.441636
5     -0.087450  0.010971       7        1   7.451153
4     -0.132364 -0.042027       8        1   6.702174
3      0.134968 -0.015412       9        1   6.255577
7      0.033435 -0.186488      10        1   1.217377, topic_info=          Term          Freq         Total Category  logprob  loglift
481      trump   7486.000000   7486.000000  Default  30.0000  30.0000
436      state  10054.000000  10054.000000  Default  29.0000  29.0000
231    islamic   6078.000000   6078.000000  Default  28.0000  28.0000
44      attack   3561.000000   3561.000000  Default  27.0000  27.0000
295        new   5957.000000   5957.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
104  copyright     30.904641   1451.559394  Topic10  -6.0352   0.5590
770       city     28.307025   2463.252230  Topic10  -6.1230  -0.0577
63      border     25.982221    833.567393  Topic10  -6.2087   0.9402
396        say     26.375908   2268.917086  Topic10  -6.1937  -0.0461
156    english     25.794628   1649.724039  Topic10  -6.2159   0.2503

[868 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
25466      6  0.969129      abaaoud
897        1  0.943825        abadi
897        4  0.026813        abadi
897        7  0.026813        abadi
25467      7  0.963280      abballa
...      ...       ...          ...
36487      6  0.917560       ziadeh
32291      9  0.897275         zuck
4617       3  0.015814   zuckerberg
4617       9  0.972569   zuckerberg
11345      9  0.944427  zuckerbergs

[3160 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 9, 3, 10, 7, 6, 5, 4, 8])